In [5]:
search_for = 121
start_from = 0
threads = 10
thread = 5
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0041391849517822266
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 15
all cases: 237758115
type: [1, 1, 1, 1, 121] 121
cases of this type: 214358881
100000 11.500440756549029
200000 20.49974715585808
300000 18.28563991059628
400000 16.918595768591366
500000 19.763814463643833
600000 17.94533958038203
700000 18.40659222639547
800000 16.78713569216704
900000 10.457207693652014
1000000 13.76670275574666
1100000 19.35676693638073
1200000 19.48495081172067
1300000 16.351493050985702
1400000 18.94360645216355
1500000 17.63649293217366
1600000 17.53942208176151
1700000 17.43608413322726
1800000 9.050492481373071
1900000 16.856191145364974
2000000 20.386214820102754
2100000 20.071271998605788
2200000 20.844873637399207
2300000 18.758310098687684
2400000 18.88918639406258
2500000 19.727980202317582
2600000 18.50130871083901
2700000 8.062963664524426
2800000 21.11200224846778
2900000 17.851795531121066
3000000 19.106

29500000 15.16652756737371
29600000 15.19685092348199
29700000 16.00923067626023
29800000 16.420191957158462
29900000 9.334237221814565
30000000 15.630479366780072
30100000 16.842635622678873
30200000 16.08892957551821
30300000 16.703930180262148
30400000 8.917645755565216
30500000 16.537389524039423
30600000 15.667571403639883
30700000 15.32559496154159
30800000 15.708372096892479
30900000 14.022675222865015
31000000 11.336695495351842
31100000 10.161808178234622
31200000 14.718816490983903
31300000 16.038451008782296
31400000 15.060614630000485
31500000 16.363537846346567
31600000 16.204372996923464
31700000 9.377908966163082
31800000 16.302705855348677
31900000 17.113474138793457
32000000 16.529250543979245
32100000 16.27582889213258
32200000 7.737249661358731
32300000 16.554224679050503
32400000 15.093333086390425
32500000 15.977891370308889
32600000 15.765096306926187
32700000 14.602439387373806
32800000 7.054825856835848
32900000 13.315035131162293
33000000 14.431821476540556
331

59100000 11.884044996051411
59200000 12.407871251972473
59300000 12.385829582389407
59400000 5.158011580355644
59500000 12.671022298576386
59600000 11.690800016705417
59700000 11.85973522274834
59800000 7.016421357743241
59900000 12.65350181216059
60000000 11.930881882185778
60100000 13.00833355337049
60200000 13.570067209460342
60300000 12.865454733585256
60400000 13.527550742159937
60500000 13.734491409747669
60600000 12.38532863180092
60700000 12.410017674179265
60800000 5.843846705844562
60900000 11.52037052476287
61000000 12.025461952390033
61100000 10.515306910738232
61200000 6.764317543452779
61300000 11.469335426777644
61400000 11.595423694987243
61500000 8.749797745576949
61600000 8.851652174497582
61700000 12.40982757011696
61800000 12.727824816672086
61900000 12.599961481452565
62000000 13.172309983790003
62100000 13.259272507062397
62200000 13.477179028905246
62300000 13.139233486337687
62400000 12.318592601802706
62500000 11.328914044770622
62600000 6.671980805407256
62700

88800000 10.50955633857889
88900000 10.63752284907484
89000000 10.005178575767937
89100000 9.74234387221719
89200000 11.142088946981652
89300000 8.489977593588135
89400000 6.221307083821983
89500000 4.3350811637946265
89600000 8.746711524039922
89700000 6.564089025256237
89800000 10.131522475671689
89900000 10.452787072671747
90000000 9.879485826839854
90100000 10.326536467977341
90200000 10.413575965811575
90300000 11.041582881760268
90400000 11.244078205674265
90500000 12.412980120194206
90600000 9.862686616045911
90700000 10.392545571006593
90800000 9.862909681680375
90900000 9.219165217341066
91000000 9.800926836601601
91100000 7.564392259459264
91200000 7.5303304434120655
91300000 4.060954232438965
91400000 5.609523771326738
91500000 9.677243373893905
91600000 9.899292364747348
91700000 9.911723552407047
91800000 9.874361826122156
91900000 9.68174088839236
92000000 10.052448945673191
92100000 11.257861570181163
92200000 10.782170444515724
92300000 10.442135464904595
92400000 9.906

118100000 7.939036163062337
118200000 8.267299007741183
118300000 8.017907997316993
118400000 7.777626974443621
118500000 8.17105363695987
118600000 8.154574163114477
118700000 9.0404516084069
118800000 9.021931354062389
118900000 8.548978906738274
119000000 8.131270280630265
119100000 8.441672899772733
119200000 7.852938334279901
119300000 8.038218338562608
119400000 8.51874639733398
119500000 4.915105598869867
119600000 3.563394339168575
119700000 4.254023228444438
119800000 7.447290598491184
119900000 8.424457552242746
120000000 8.564419977256506
120100000 8.155117429070394
120200000 7.629167914148916
120300000 8.719924032889674
120400000 8.517951289353054
120500000 9.165798177766879
120600000 8.565670198068837
120700000 8.663135560783049
120800000 8.214376143674384
120900000 7.821542019467025
121000000 8.000113813916316
121100000 8.468165219324797
121200000 7.673269224130423
121300000 4.459485562688543
121400000 3.3546525141011276
121500000 4.716171428320706
121600000 8.25369313919

147400000 6.586093750249502
147500000 6.144337011504435
147600000 4.090250717257977
147700000 5.5875821452628145
147800000 5.923345649152862
147900000 6.22927719014296
148000000 3.235459799947573
148100000 6.119122273482418
148200000 6.2956410815791495
148300000 2.761730657610599
148400000 5.884659112108978
148500000 6.563205866512468
148600000 6.54316334797227
148700000 6.698043702146612
148800000 6.936859760435432
148900000 6.764627949320999
149000000 6.214476423931321
149100000 6.436268286565294
149200000 6.07726695816662
149300000 5.837431470608549
149400000 3.2652739317844115
149500000 5.922168941296535
149600000 5.794192929031925
149700000 4.372237230756015
149800000 4.712489452830075
149900000 6.038900550218267
150000000 5.972136555078487
150100000 2.9631191089205444
150200000 5.832730049875766
150300000 6.566679172261914
150400000 6.482527862148292
150500000 6.750016072206699
150600000 6.666786287381529
150700000 6.783337962803229
150800000 6.438993001884828
150900000 6.1484053

176600000 4.455517823328168
176700000 4.478325897103726
176800000 4.5158536010073425
176900000 2.2298454252724085
177000000 4.688213416850894
177100000 4.589671108457294
177200000 4.6717469764387936
177300000 4.822571789266368
177400000 4.397975064189543
177500000 2.7258653475183183
177600000 4.5558036074462835
177700000 4.207679461400081
177800000 4.396973320037087
177900000 3.9555910152326623
178000000 4.332533015420652
178100000 1.8151096971763239
178200000 4.070706036280533
178300000 4.006197131049941
178400000 4.627256170595924
178500000 4.3574957209768295
178600000 4.077298019222191
178700000 2.3670815736413
178800000 4.483104257140775
178900000 4.42753969805038
179000000 4.4958049681978025
179100000 4.372857555153271
179200000 3.5833005392430057
179300000 3.117936914175431
179400000 4.203740861995459
179500000 4.2709733472417994
179600000 3.9439008501679367
179700000 3.9841396581727166
179800000 3.1678748611082828
179900000 2.4126190732078383
180000000 3.7104155801332794
1801000

205400000 2.2733023773770036
205500000 1.500337929438919
205600000 1.3986836706802181
205700000 2.5684924767428807
205800000 2.5684060275405542
205900000 2.4491345941078064
206000000 2.5667504870832856
206100000 2.382947465664516
206200000 2.3556067352797885
206300000 2.461890471526444
206400000 1.4321728376919924
206500000 2.0734293846339384
206600000 2.3532361055980955
206700000 2.301512447053522
206800000 2.3744250296493936
206900000 2.587161731630077
207000000 2.421696731459856
207100000 2.3081902851392626
207200000 1.914253237109555
207300000 1.4551450675700308
207400000 1.4552349122569463
207500000 2.4409314287104706
207600000 2.3644547747143907
207700000 2.4565139009598624
207800000 2.3266656682531437
207900000 2.162420940544665
208000000 2.2041838141363232
208100000 2.4410643910291356
208200000 0.9680533589935898
208300000 2.402930614945541
208400000 2.380135603847242
208500000 2.2337668858159088
208600000 2.172724807900422
208700000 2.3270118539608124
208800000 2.2563115816440

233400000 0.12147389888679981
233500000 0.10197217281092538
233600000 0.10591727836048603
233700000 0.10899215373835962
233800000 0.11089064549055364
best so far: 0
type: [1, 1, 1, 121, 1] 121
cases of this type: 1771561
233900000 0.06926625047399601
234000000 0.02109194696884354
234100000 0.023633285009748407
234200000 0.01972967310572333
234300000 0.02176039509800077
234400000 0.021166324900137055
234500000 0.02079389555816518
234600000 0.019905826152563092
234700000 0.01950649133042826
234800000 0.01865483280987541
234900000 0.01784431291655699
235000000 0.017433318994571765
235100000 0.01695253056954344
235200000 0.01351415578749776
235300000 0.009569337229884333
235400000 0.015409852692269617
235500000 0.014710354635864497
235600000 0.013526961784538296
best so far: 0
type: [1, 1, 11, 1, 11] 121
cases of this type: 1771561
235700000 0.05305157276363174
235800000 0.060108970748672885
235900000 0.04777476750603649
236000000 0.03378940513310168
236100000 0.04305385181578994
236200000